In [1]:
import pandas as pd
import numpy as np 
import glob 
import warnings 
from collections import Counter
warnings.filterwarnings("ignore")
import plotly.express as px 
import seaborn as sns 
import matplotlib.pyplot as plt 
import lightgbm as lgbm 
from sklearn.model_selection import StratifiedKFold, train_test_split
import math
import os 
import random
import torch 
import pyarrow as pa
import pyarrow.parquet as pq
import torch.nn as nn
#from transformers import AdamW
from torch.utils.data import Dataset , DataLoader
from colorama import Fore , Style
r__=Fore.RED
g__=Fore.GREEN
st__=Style.RESET_ALL

In [2]:
#Split training data into 80% train + 20% test

train = pd.read_csv("optiver_raw_data/train.csv")

time_id_of_first_80_percent = train.iloc[18384,1]


In [13]:
stock_0 = train[train["stock_id"]==0]
df_train = pd.DataFrame()
df_test = pd.DataFrame()
for i in range(127):
    stock_i = train[train["stock_id"]==i]

    stock_train = stock_i[stock_i['time_id'] <= time_id_of_first_80_percent]
    stock_test = stock_i[stock_i['time_id'] > time_id_of_first_80_percent]

    df_train = pd.concat([df_train, stock_train])
    df_test = pd.concat([df_test, stock_test])

df_train.to_parquet('target_data/target_train.parquet')
df_test.to_parquet('target_data/target_test.parquet')

In [14]:
#Split book training data into 80% train + 20% test for stocks 0-5
for i in range(127):
    filename = "optiver_raw_data/book_train.parquet/stock_id=" + str(i)
    if not os.path.exists(filename):
        continue
    book_current_stock = pd.read_parquet(filename)

    stock_0_train = book_current_stock[book_current_stock['time_id'] <= time_id_of_first_80_percent]
    stock_0_train.to_parquet('stock_train/stock_' + str(i) + '_train.parquet')

    stock_0_test = book_current_stock[book_current_stock['time_id'] > time_id_of_first_80_percent]
    stock_0_test.to_parquet('stock_test/stock_' + str(i) + '_test.parquet')


In [12]:
#Split trade training data into 80% train + 20% test for stocks 0-5
for i in range(127):
    filename = "optiver_raw_data/trade_train.parquet/stock_id=" + str(i)
    if not os.path.exists(filename):
        continue
    trade_current_stock = pd.read_parquet(filename)

    stock_train = trade_current_stock[trade_current_stock['time_id'] <= time_id_of_first_80_percent]
    stock_train.to_parquet('stock_trade_train/stock_' + str(i) + '_train.parquet')

    stock_test = trade_current_stock[trade_current_stock['time_id'] > time_id_of_first_80_percent]
    stock_test.to_parquet('stock_trade_test/stock_' + str(i) + '_test.parquet')

In [5]:
trade_train_stock_id_0 = pd.read_parquet("optiver_raw_data/trade_train.parquet/stock_id=0")
trade_train_stock_id_0

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1
...,...,...,...,...,...
123438,32767,471,0.998659,200,3
123439,32767,517,0.998515,90,1
123440,32767,523,0.998563,1,1
123441,32767,542,0.998803,90,4


In [11]:
trade_train_stock_id_0.iloc[99230:99231, 0:]

,time_id,seconds_in_bucket,price,size,order_count
99230,25680,547,0.998826,32,2


In [6]:
trade_train_stock_id_0[trade_train_stock_id_0["time_id"] == time_id_of_first_80_percent]

,time_id,seconds_in_bucket,price,size,order_count
99231,25683,58,1.001427,200,5
99232,25683,98,0.999538,207,7
99233,25683,120,1.000157,416,12
99234,25683,125,1.000125,21,2
99235,25683,158,1.000876,66,1
99236,25683,258,1.001582,291,11
99237,25683,261,1.001427,2,1
99238,25683,275,1.001127,1,1
99239,25683,278,1.000239,248,7
99240,25683,292,1.002078,101,3
